In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta, date
from ast import literal_eval

'''This script produces a file which shows transactions. Each row is a transaction
with arrival, depart, and room type.'''

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# load purchase data
purchases_df = pd.read_csv('../../../data/cabot_data/clean_base/2018_clean.csv',
                           parse_dates=['ARRIVAL', 'DEPART', 'LOOK_DATE'])
purchases_df = (purchases_df[purchases_df['CANCEL_INDICATOR'] == 0] # look at bookings only
                .drop(['RESNO', 'CANCEL_INDICATOR'], axis=1)
                .groupby('group_id').first() # collapse groups
                .sort_values('LOOK_DATE'))

# add stay len col to df_data
purchases_df['stay_len'] = (purchases_df['DEPART'] - purchases_df['ARRIVAL']).dt.days
# anything greater than 4 is 4
purchases_df['stay_len'] = purchases_df['stay_len'].apply(lambda x: 4 if x > 4 else x)

# initialize some helper vars
df_grouped = purchases_df.groupby(by=['LOOK_DATE']).count()
periods_per_day = df_grouped['DEPART'].max()

look_start = purchases_df['LOOK_DATE'].min()
look_end = purchases_df['LOOK_DATE'].max()

ssn_start = purchases_df['ARRIVAL'].min()
ssn_end = purchases_df['ARRIVAL'].max()

In [ ]:
# build empty trans df
intraday_range = range(1, periods_per_day+1)
look_range = pd.date_range(look_start, look_end)
ssn_range = pd.date_range(ssn_start, ssn_end)

trans_df = pd.DataFrame(index = pd.MultiIndex.from_product([look_range, intraday_range], 
                                                           names=['LOOK_DATE', 'INTRADAY']),
                        columns=['PRODUCT'])

In [ ]:
# fill in purchases
for index, row in purchases_df.iterrows():
    
    # helper vars
    unit = row['UNIT']
    week = row['ARRIVAL'].week
    dow = row['ARRIVAL'].dayofweek + 1
    stay_len = row['stay_len']
    
    intra_day_counter = 1 # counter var for intra day
    cur_cell = (row['LOOK_DATE'], intra_day_counter)
    while(trans_df.loc[cur_cell].notnull()[0]): # find empty cell
        intra_day_counter += 1
        cur_cell = (row['LOOK_DATE'], intra_day_counter)
    trans_df.loc[cur_cell] = [(row['UNIT'], week, dow, stay_len)]
    print(row['LOOK_DATE'])

In [ ]:
trans_df.to_csv('../../../data/cabot_data/sprint_2/trans_s2_raw.csv')

In [ ]:
trans_df_t = pd.read_csv('../../../data/cabot_data/sprint_2/trans_s2_raw.csv', index_col=['LOOK_DATE', 'INTRADAY'])
prod_num_map = pd.read_csv('../../../data/cabot_data/sprint_2/prod_num_map.csv',
                           header=None,
                           index_col=[0,1,2,3])

# import product number mapping
prod_num_map = pd.Series(prod_num_map.iloc[:,0])
prod_num_map = prod_num_map.to_dict()

# convert product strings to tuples, and map
trans_df_t = trans_df_t.fillna('0')
trans_df_t['PRODUCT'] = [literal_eval(x) for x in trans_df_t['PRODUCT']]
trans_df_t['PRODUCT'] = trans_df_t['PRODUCT'].map(prod_num_map)

# misc processing
trans_df_t = trans_df_t.fillna(0).astype(int)
trans_df_t = trans_df_t.reset_index()
trans_df_t = trans_df_t.drop(['LOOK_DATE', 'INTRADAY'], axis=1)
trans_df_t.index += 1

# save
trans_df_t.to_csv('../../../data/cabot_data/sprint_2/trans_s2.csv')

In [153]:
trans_df

PRODUCT
LOOK_DATE  INTRADAY                 
2018-01-02 1          (CD, 32, 4, 2)
           2          (DD, 25, 4, 2)
           3          (DD, 25, 4, 2)
           4         (DKB, 39, 4, 3)
           5          (DK, 36, 4, 1)
           6         (DKB, 39, 4, 3)
           7         (4BV, 36, 4, 3)
           8         (4BV, 21, 5, 4)
           9          (DD, 32, 4, 2)
           10         (CK, 25, 4, 2)
           11         (CK, 20, 5, 3)
           12         (DK, 36, 4, 1)
           13         (DK, 32, 4, 1)
           14         (CD, 35, 3, 4)
           15        (4BV, 36, 4, 3)
           16         (DD, 25, 4, 2)
           17         (DK, 25, 4, 2)
           18                    NaN
           19                    NaN
           20                    NaN
           21                    NaN
           22                    NaN
           23                    NaN
           24                    NaN
           25                    NaN
           26                    NaN
           27                    NaN
           28                    NaN
           29                    NaN
           30                    NaN
...                              ...
2018-10-27 52                    NaN
           53                    NaN
           54                    NaN
           55                    NaN
           56                    NaN
           57                    NaN
           58                    NaN
           59                    NaN
           60                    NaN
           61                    NaN
           62                    NaN
           63                    NaN
           64                    NaN
           65                    NaN
           66                    NaN
           67                    NaN
           68                    NaN
           69                    NaN
           70                    NaN
           71                    NaN
           72                    NaN
           73                    NaN
           74                    NaN
           75                    NaN
           76                    NaN
           77                    NaN
           78                    NaN
           79                    NaN
           80                    NaN
           81                    NaN

[24219 rows x 1 columns]